### Chen, Cheng, Liu, and Tang (2025)

Seho Jeong, Sogang University

References
- **Chen, Hui, Yuhan Cheng, Yanchu Liu, and Ke Tang. 2025.** "Teaching Economics to the Machines." Available at SSRN: https://ssrn.com/abstract=4642167.
- **Lian, Tony. 2020.** "Webscraping Options Data with Python and YFinance." https://medium.com/@txlian13/webscrapping-options-data-with-python-and-yfinance-e4deb0124613.

In [25]:
import numpy as np
import pandas as pd
from scipy.stats import norm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns

import yfinance as yf

import datetime
from tqdm.auto import tqdm

In [3]:
# Set custom color map.
cm = ['#116FA1', '#2DC0D2', '#E0B266', '#00979F', '#70CAC6', '#005A7D']
bg = '#F9F9F9'

In [4]:
# Set sample date.
START_DATE = '2000-01-02'
END_DATE = '2023-03-31'

#### Data Collection

#### Synthetic Data Generation

In [17]:
n_samples = 800_000                   # sample size
gen = np.random.default_rng(seed=101) # random generator

# Generate feature matrix.
features = {
    'S':        gen.uniform(0.0, 5.0, n_samples),
    'K':        gen.uniform(0.0, 5.0, n_samples),
    'T':        gen.uniform(0.0, 4.0, n_samples),
    'r':        gen.uniform(0.0, 0.1, n_samples),
    'd':        gen.uniform(0.0, 0.1, n_samples),
    'IV1':      gen.uniform(0.0, 1.0, n_samples),
    'mom1w':    gen.uniform(-10.0, 10.0, n_samples),
    'mom4w':    gen.uniform(-10.0, 10.0, n_samples),
    'hv1m':     gen.uniform(0.0, 0.8, n_samples),
    'hv9m':     gen.uniform(0.0, 0.8, n_samples),
    'volume1m': gen.uniform(-4.0, 4.0, n_samples),
    'volume5d': gen.uniform(-4.0, 4.0, n_samples),
    'PCratio':  gen.uniform(0.0, 2.5, n_samples),
    'epratio':  gen.uniform(0.0, 0.1, n_samples),
    'spmom1w':  gen.uniform(-0.3, 0.2, n_samples),
    'spmom4w':  gen.uniform(-0.3, 0.2, n_samples)
}

df = pd.DataFrame(features)

# Compute Black-Scholes price, delta, and vega.
eps = 1e-8
d1 = (np.log(df.S / df.K + eps) + (df.r - df.d + 0.5 * df.IV1 ** 2) * df['T']) / (df.IV1 * np.sqrt(df['T'] + eps))
d2 = d1 - df.IV1 * np.sqrt(df['T'] + eps)

df['priceBS'] = np.exp(-df.d * df['T']) * df.S * norm.cdf(d1) - np.exp(-df.r * df['T']) * df.K * norm.cdf(d2)
df['deltaBS'] = np.exp(-df.d * df['T']) * norm.cdf(d1)
df['vegaBS'] = df.S * np.exp(-df.d * df['T']) * norm.cdf(d1) * np.sqrt(df['T'])

In [20]:
df.head()

,S,K,T,r,d,IV1,mom1w,mom4w,hv1m,hv9m,volume1m,volume5d,PCratio,epratio,spmom1w,spmom4w,priceBS,deltaBS,vegaBS
0,4.717663,2.477987,0.186826,0.034206,0.059189,0.918058,-7.420850,6.197590,0.107487,0.632580,-2.282261,3.216121,1.249736,0.043317,-0.212898,0.012198,2.233611e+00,9.541823e-01,1.945705e+00
1,1.797105,0.411058,3.354841,0.068683,0.074423,0.478771,-4.965821,4.214344,0.412847,0.699477,2.790544,1.733956,1.708133,0.026323,-0.080419,-0.006099,1.084690e+00,7.650922e-01,2.518392e+00
2,3.924027,0.936588,1.635719,0.041890,0.081608,0.727274,6.139472,8.023063,0.413223,0.093650,-3.556227,3.601670,1.799108,0.049213,-0.174566,-0.165112,2.605865e+00,8.518777e-01,4.275270e+00
3,2.956391,0.278929,1.079132,0.062159,0.080088,0.168121,-3.489696,-2.293017,0.588792,0.100307,-0.985116,-0.784017,2.461095,0.053302,-0.207402,-0.286155,2.450779e+00,9.172034e-01,2.816857e+00
4,1.471643,3.086585,3.126482,0.031806,0.014201,0.026067,-3.061103,9.615902,0.774026,0.022502,1.541858,1.399255,1.575107,0.086683,0.074949,-0.216808,1.439423e-52,3.189907e-50,8.300580e-50


In [23]:
len(df.columns)

19

#### Source Domain

In [24]:
class ResMLP(nn.Module):
    
    def __init__(self, input_dim=16, hidden_dim=22, n_res_layers=16):
        super(ResMLP, self).__init__()

        # Input projection
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        # Residual layers
        self.res_layers = nn.ModuleList([
            nn.Linear(hidden_dim, hidden_dim) for _ in range(n_res_layers)
        ])

        # Output projection
        self.output_layer = nn.Linear(hidden_dim, 1)

        # Activation
        self.act = nn.LeakyReLU()

    def forward(self, x):
        x = self.act(self.input_layer(x)) # (batch_size, 16) -> (batch_size, 22)

        # Residual blocks
        for layer in self.res_layers:
            out = self.act(layer(x))
            x = x + out

        # Final scalar output
        price = self.output_layer(x) # -> (batch_size, 1)

        return price

In [36]:
# Set seeds for reproductibility.
seed = 101
torch.manual_seed(seed)
np.random.seed(seed)

# Prepare data tensors.
data = df.to_numpy(dtype=np.float32)
X = data[:, :16]
y_price = data[:, 16:17]
y_delta = data[:, 17:18]
y_vega = data[:, 18:19]

# Create DataLoader with 600 batches.
dataset = TensorDataset(torch.from_numpy(X),
                        torch.from_numpy(y_price),
                        torch.from_numpy(y_delta),
                        torch.from_numpy(y_vega))
loader = DataLoader(dataset, batch_size=len(dataset)//600, shuffle=True, drop_last=True)

# Define the model.
model = ResMLP()
model.train()

# Set optimizer, loss weights, and epsilons.
optimizer = optim.Adam(model.parameters(), lr=2e-4)
w1, w2, w3 = 0.2, 0.2, 1.0
eps_c = 1e-3

# training loop
for epoch in tqdm(range(1, 201)):

    epoch_loss = 0.0

    for Xb, y_pb, y_d, y_v in loader:

        optimizer.zero_grad()

        p_pred = model(Xb)

        loss_delta = torch.abs(p_pred.grad_fn is None and 0 or torch.zeros_like(y_d))

        mae_delta = torch.mean(torch.abs(p_pred * 0 + y_d - y_d))
        mae_vega = torch.mean(torch.abs(p_pred * 0 + y_v - y_v))
        weights = 1.0 / (y_d.abs() + eps_c)
        wmae_price = torch.mean(weights * torch.abs(p_pred - y_pb))

        loss = w1 * mae_delta + w2 * mae_vega + w3 * wmae_price
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    print(f'Epoch {epoch:3d} / 200 - Loss: {epoch_loss / len(loader):.6f}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch   1 / 200 - Loss: 64.280435
Epoch   2 / 200 - Loss: 4.887365
Epoch   3 / 200 - Loss: 3.797798
Epoch   4 / 200 - Loss: 3.473552
Epoch   5 / 200 - Loss: 3.094951
Epoch   6 / 200 - Loss: 2.789109
Epoch   7 / 200 - Loss: 2.558269
Epoch   8 / 200 - Loss: 2.449376
Epoch   9 / 200 - Loss: 2.266058
Epoch  10 / 200 - Loss: 2.096360
Epoch  11 / 200 - Loss: 1.951326
Epoch  12 / 200 - Loss: 1.859161
Epoch  13 / 200 - Loss: 1.743374
Epoch  14 / 200 - Loss: 1.659496
Epoch  15 / 200 - Loss: 1.571539
Epoch  16 / 200 - Loss: 1.586675
Epoch  17 / 200 - Loss: 1.642082
Epoch  18 / 200 - Loss: 1.588086
Epoch  19 / 200 - Loss: 1.650443
Epoch  20 / 200 - Loss: 1.577456
Epoch  21 / 200 - Loss: 1.552106
Epoch  22 / 200 - Loss: 1.398402
Epoch  23 / 200 - Loss: 0.708075
Epoch  24 / 200 - Loss: 0.675405
Epoch  25 / 200 - Loss: 0.696502
Epoch  26 / 200 - Loss: 0.657837
Epoch  27 / 200 - Loss: 0.567989
Epoch  28 / 200 - Loss: 0.584863
Epoch  29 / 200 - Loss: 0.561166
Epoch  30 / 200 - Loss: 0.574482
Epoch  31

In [37]:
model

ResMLP(
  (input_layer): Linear(in_features=16, out_features=22, bias=True)
  (res_layers): ModuleList(
    (0-15): 16 x Linear(in_features=22, out_features=22, bias=True)
  )
  (output_layer): Linear(in_features=22, out_features=1, bias=True)
  (act): LeakyReLU(negative_slope=0.01)
)